In [55]:
from langchain_gigachat.chat_models import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableSequence
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import json
import re
pd.set_option('display.max_colwidth', 1000)

In [5]:
llm = GigaChat(
credentials="OGRmN2Q5OWEtOTQ4Mi00N2FmLWFmNjgtYTYwZGI5ZmZmYjhmOmVjOTkwNGZkLTFlOTctNGZhYy05MmY2LWViZDJhMDIyNjBhZA==",
scope="GIGACHAT_API_PERS", # или GIGACHAT_API_CORP для юридических лиц
model="GigaChat-2", # можно указать конкретную версию, по умолчанию Lite
verify_ssl_certs=False,
streaming=True, # включение потоковой передачи
temperature=0.2, # настройка креативности
max_tokens=1000 # максимальная длина ответа
)
# Проверка подключения
response = llm.invoke("Привет! Как дела?")
print(response.content)

Привет! Всё отлично, готов общаться и помогать тебе с любыми вопросами. А ты как настроение держишь сегодня?


In [71]:
df1 = pd.read_csv(r'rental_applications_amount.csv') # С разметкой
df2 = pd.read_csv(r'rental_applications_text.csv', delimiter=';') # Без разметки
df1_to_test_= df1[:20] # 20 наблюдений для проверки количества взрослых/детей
df1_to_test_['count_adults'] = [1, 5, 4, 2, 3, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2, 6]
df1_to_test_['count_children'] = [0, 0, 3,  1, 3, 0, 0, 2, 1, 2, 2, 1, 3, 3, 4, 0, 0, 2, 0, 0]

C:\Users\gromo\AppData\Local\Temp\ipykernel_5980\4050383508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_to_test_['count_adults'] = [1, 5, 4, 2, 3, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 4, 2, 2, 2, 6]
C:\Users\gromo\AppData\Local\Temp\ipykernel_5980\4050383508.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_to_test_['count_children'] = [0, 0, 3,  1, 3, 0, 0, 2, 1, 2, 2, 1, 3, 3, 4, 0, 0, 2, 0, 0]


In [72]:
json_schema = {
    "type": "object",
    "properties": {
        "total_persons": {"type": "integer"},
        "count_adults": {"type": "integer"},
        "count_children": {"type": "integer"}
    },
    "required": ["total_persons", "count_adults", "count_children"],
    "additionalProperties": False
}

In [73]:
# Простой промпт для извлечения количества людей
system_message = SystemMessage(content="""
ы - эксперт по анализу текстовых заявок на аренду жилья. Проанализируй следующий текст заявки на аренду жилья и извлеки количество человек, которые будут проживать.

Текст заявки: {text}

Верни 3 числа (целые числа), соответствующее количеству проживающих  (для всех записей), количеству проживающих взрослых людей (для первых 20 записей), количество проживающих детей (для первых 20 записей).
Если количество не указано явно, постарайся определить его по контексту, где не указан возраст ребенка или явного указания их количества то значит в заявки нет детей.

Запиши в json-схему по очереди total_persons - общее количество людей, count_adults - количество взрослых людей, count_children - количество детей:"""
)

human_template = """Текст заявки: {text}

Проанализируй текст и верни JSON с количеством проживающих:"""

In [74]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_message.content),
    ("human", human_template)
])

In [75]:
# Создание цепочки
chain = prompt | llm

# Функция для извлечения JSON из ответа
def extract_json_from_response(response):
    try:
        json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
        if json_match:
            return json.loads(json_match.group())
        else:
            return json.loads(response.content)
    except json.JSONDecodeError:
        print(f"Ошибка парсинга JSON: {response.content}")
        return {"total_persons": None, "count_adults": None, "count_children": None}

In [76]:
# Обработка данных и создание результатов
results = []

for idx, row in df2.iterrows():
    try:
        # Получение ответа от модели
        response = chain.invoke({"text": row['text']})
        
        # Извлечение JSON
        result_json = extract_json_from_response(response)
        
        # Поиск соответствующей разметки
        true_data = df1[df1['text_id'] == row['text_id']]
        true_amount = true_data['amount'].values[0] if not true_data.empty else None
        
        # Создание записи результата
        result_row = {
            'text_id': row['text_id'],
            'predicted_total_persons': result_json.get('total_persons'),
            'true_total_persons': true_amount,
            'correct': result_json.get('total_persons') == true_amount if true_amount is not None else None,
            'predicted_adults': result_json.get('count_adults'),
            'predicted_children': result_json.get('count_children')
        }
        
        results.append(result_row)
        
        print(f"Обработана запись {idx+1}/{len(df2)}: {result_row}")
        
    except Exception as e:
        print(f"Ошибка при обработке text_id {row['text_id']}: {str(e)}")
        results.append({
            'text_id': row['text_id'],
            'predicted_total_persons': None,
            'true_total_persons': None,
            'correct': None,
            'predicted_adults': None,
            'predicted_children': None
        })

Обработана запись 1/100: {'text_id': 14205200, 'predicted_total_persons': 1, 'true_total_persons': np.int64(1), 'correct': np.True_, 'predicted_adults': 1, 'predicted_children': 0}
Обработана запись 2/100: {'text_id': 319097075, 'predicted_total_persons': 8, 'true_total_persons': np.int64(5), 'correct': np.False_, 'predicted_adults': 5, 'predicted_children': 3}
Обработана запись 3/100: {'text_id': 98881311, 'predicted_total_persons': 7, 'true_total_persons': np.int64(7), 'correct': np.True_, 'predicted_adults': 4, 'predicted_children': 3}
Обработана запись 4/100: {'text_id': 44587027, 'predicted_total_persons': 6, 'true_total_persons': np.int64(6), 'correct': np.True_, 'predicted_adults': 4, 'predicted_children': 2}
Обработана запись 5/100: {'text_id': 352802829, 'predicted_total_persons': 6, 'true_total_persons': np.int64(6), 'correct': np.True_, 'predicted_adults': 3, 'predicted_children': 3}
Обработана запись 6/100: {'text_id': 52157605, 'predicted_total_persons': 2, 'true_total_per

In [77]:
results_df = pd.DataFrame(results)
# Сохранение результатов
results_df.to_csv('results.csv', index=False, encoding='utf-8')

In [78]:
results_df

,text_id,predicted_total_persons,true_total_persons,correct,predicted_adults,predicted_children
0,14205200,1,1,True,1.0,0.0
1,319097075,8,5,False,5.0,3.0
2,98881311,7,7,True,4.0,3.0
3,44587027,6,6,True,4.0,2.0
4,352802829,6,6,True,3.0,3.0
...,...,...,...,...,...,...
95,233661411,9,9,True,4.0,5.0
96,77792435,1,1,True,1.0,0.0
97,102311098,2,2,True,2.0,0.0
98,123993850,2,2,True,NaN,NaN


In [79]:
print("\nПримеры JSON ответов для 5 заявок:")
for i in range(min(5, len(results))):
    print(f"Заявка {i+1}: {results[i]}")


Примеры JSON ответов для 5 заявок:
Заявка 1: {'text_id': 14205200, 'predicted_total_persons': 1, 'true_total_persons': np.int64(1), 'correct': np.True_, 'predicted_adults': 1, 'predicted_children': 0}
Заявка 2: {'text_id': 319097075, 'predicted_total_persons': 8, 'true_total_persons': np.int64(5), 'correct': np.False_, 'predicted_adults': 5, 'predicted_children': 3}
Заявка 3: {'text_id': 98881311, 'predicted_total_persons': 7, 'true_total_persons': np.int64(7), 'correct': np.True_, 'predicted_adults': 4, 'predicted_children': 3}
Заявка 4: {'text_id': 44587027, 'predicted_total_persons': 6, 'true_total_persons': np.int64(6), 'correct': np.True_, 'predicted_adults': 4, 'predicted_children': 2}
Заявка 5: {'text_id': 352802829, 'predicted_total_persons': 6, 'true_total_persons': np.int64(6), 'correct': np.True_, 'predicted_adults': 3, 'predicted_children': 3}


In [97]:
valid_results = results_df.dropna(subset=['predicted_total_persons', 'true_total_persons'])
accuracy_true_total_person = accuracy_score(valid_results['true_total_persons'], valid_results['predicted_total_persons'])

print("Точность по общему количеству людей:",round(accuracy_true_total_person, 2))

Точность по общему количеству людей: 0.98


In [95]:
accuracy_adults = accuracy_score(df1_to_test_['count_adults'][:19], results_df['predicted_adults'][:19])

print("Точность по взрослым:",round(accuracy_adults, 2))

Точность по взрослым: 0.95


In [96]:
accuracy_children = accuracy_score(df1_to_test_['count_children'][:19], results_df['predicted_children'][:19])

print("Точность по детям:",round(accuracy_children, 2))

Точность по детям: 0.89
